In [6]:
import pandas as pd
import numpy as np
import os
import openpyxl

In [15]:
def clean_jams(input_file: str, rank: str = "Genus", input_type="csv"):
    df = pd.DataFrame()
    if input_type == "csv":
        df = pd.read_csv(input_file, index_col=0)
    elif input_type == "excel":
        df = pd.read_excel(input_file, index_col=0)
    else:
        raise Exception("Input type not recognized.")
    
    df["Count"] = df["NumBases"] / df["NumBases"].sum()
    # display(df.head())
    genus_df = df[[rank, "Count"]].groupby(rank).sum()
    genus_df.sort_values("Count", ascending=False, inplace=True)

    # We need to remove g__ and s__ from the index names
    genus_df.index = genus_df.index.str.replace("g__", "")
    genus_df.index = genus_df.index.str.replace("s__", "")

    output_dir = os.path.dirname(input_file)
    file_name = os.path.basename(input_file).split(".")[0]

    genus_df.to_csv(os.path.join(output_dir, f"{file_name}_{rank.lower()}_relabund.csv"))

# clean_jams("pipelines/camisimGI/jams/s1.csv", "Genus")

In [46]:
def clean_jams_beta(input_file: str, rank="genus", output_dir=""):
    """Clean JAMS output excel file and save them in the preferred format.
    
    Parameters
    ----------
    input_dir : str
        Path to directory containing JAMS output files.
    rank : str
        Taxonomic rank.
    """

    df = pd.read_excel(input_file, index_col=0, sheet_name=1)
    # Make everything into relative abundances
    df = df / df.sum(axis=0)

    # We need to find anything with s__ or g__ in the row.names.
    df.reset_index(inplace=True)

    # If s__ is in the name, we need to get the genus name.
    index_names = []
    if rank == "genus":
        df = df.where(df["row.names"].str.contains("g__|s__")).dropna()
        lkt = df["row.names"].to_list()
        for i in lkt:
            if "s__" in i:
                index_names.append(i.split("s__")[1].split("_")[0])
            else:
                index_names.append(i.split("g__")[1].split("_")[0])

    elif rank == "species":
        df = df.where(df["row.names"].str.contains("s__")).dropna()
        lkt = df["row.names"].to_list()
        for i in lkt:
            index_names.append(i.split("s__")[1])
            
        index_names = [i.replace("_", " ") for i in index_names]


    df.index = index_names
    df.drop(columns=["row.names"], inplace=True)

    # Sum all of the rows with the same index name.
    df = df.groupby(df.index).sum()

    # Save each column as a separate file.
    for col in df.columns:
        df[col].to_csv(os.path.join(output_dir, f"{col}_{rank}_relabund.csv"), index_label=f"{rank}")

clean_jams_beta("/Volumes/TBHD_share/valencia/pipelines/microbio_spectrum/nextseq/jamsbeta/toulousse_Relabund_PPM.xlsx", rank="genus", output_dir="pipelines/toulousse/jams")
